In [ ]:
import pandas as pd
import numpy as np

from src.all_in_one import *

In [ ]:
processed_data_path = path_manager.processed_data

In [ ]:
rid = pd.DataFrame()
if os.path.exists(processed_data_path) and os.path.isdir(processed_data_path):
    for file_name in os.listdir(processed_data_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(processed_data_path, file_name)
            df = pd.read_csv(file_path)
            df_rid = df['rid']
            rid = pd.concat([rid, df_rid])

In [ ]:
rid = rid.drop_duplicates(keep='first')
rid = rid.sort_values(by='rid', ascending=True).reset_index(drop=True)

In [ ]:
scr_file = path_manager.get_data_file(Category.PS, PSFile.SCR)
scr_dataset = Dataset(config_file, scr_file.sheet)
scr = get_df(scr_file.path, scr_file.sheet, config_file).sort_values(by='sc_s1', ascending=True).reset_index(drop=True)

In [ ]:
scr_0 = scr.drop(columns=['scspi','nat', 'need_mhpss', 'need_trw', 'need_out_ref', 'need_tmh', 'need_pei', 'note','pei_pre_as', 'sc_re']).dropna(subset=['sc_s1']).sort_values(by='sc_s1')  # Screening basic session. Records = 705
re_scr = scr[['rid','sc_re']].dropna()  # Screening re. Records = 41
pei_pre_as = scr[['rid','pei_pre_as']].dropna()  # PEI Assessment. Records = 36

In [ ]:
pei_pre_as

In [ ]:
re_scr

In [ ]:
scr_0

In [ ]:
service_dataset = pd.merge(rid, scr, on='rid', how='left')

In [ ]:
# service_dataset

In [ ]:
scr_rid = scr[['rid', 'sc_s1']]
scr_rid['is_duplicated'] = scr_rid['rid'].duplicated(keep=False)
scr_rid['is_NaT'] = scr_rid['sc_s1'].isna()

In [ ]:
conditions = [
    (scr_rid['is_duplicated'] & ~scr_rid['is_NaT']),
    (scr_rid['is_duplicated'] & scr_rid['is_NaT']),
    (~scr_rid['is_duplicated'])
]

choices = [1, 2, 0]

scr_rid['duplicated'] = np.select(conditions, choices, default=2)

In [ ]:
scr_rid

In [ ]:
from transliterate import translit
import arabic_reshaper
from bidi.algorithm import get_display

In [ ]:
# Simple transliteration dictionary
transliteration_dict = {
    'ا': 'a', 'ب': 'b', 'ت': 't', 'ث': 'th', 'ج': 'j', 'ح': 'h', 'خ': 'kh',
    'د': 'd', 'ذ': 'dh', 'ر': 'r', 'ز': 'z', 'س': 's', 'ش': 'sh', 'ص': 's',
    'ض': 'd', 'ط': 't', 'ظ': 'z', 'ع': 'a', 'غ': 'gh', 'ف': 'f', 'ق': 'q',
    'ك': 'k', 'ل': 'l', 'م': 'm', 'ن': 'n', 'ه': 'h', 'و': 'w', 'ي': 'y',
    'ء': "'", 'آ': 'a', 'أ': 'a', 'ؤ': 'u', 'ئ': 'i', 'ى': 'a', 'ة': 'h'
}

In [ ]:
data = {
    'First Name': [
        "فاطمة", "علي", "محمد"
    ],
    'Last Name': [
        "عبدالله", " حسين", "أحمد"
    ]
}

df = pd.DataFrame(data)


def transliterate_arabic(name):
    reshaped_text = arabic_reshaper.reshape(name)
    bidi_text = get_display(reshaped_text)
    transliterated_text = ''.join(transliteration_dict.get(char, char) for char in bidi_text)
    return translit(name, 'ar', reversed=True)

df['First Name (Latin)'] = df['First Name'].apply(transliterate_arabic)
df['Last Name (Latin)'] = df['Last Name'].apply(transliterate_arabic)

df